In [1]:
import pandas as pd
import requests
import selenium
import time
import pymysql
import json
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Setup Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options = options)
driver.get("https://api.ipify.org/?format=json")
html = driver.page_source
soup = BeautifulSoup(html)
ip = soup.find('pre').text
print('IP from scrape ', ip)

IP from scrape  {"ip":"35.221.181.94"}


In [3]:
def aconcept_category():
    aconcept = requests.get('https://aconcept-vn.com/en')
    soup = BeautifulSoup(aconcept.text, 'html.parser')
    soup = soup.body.find_all('li', {'class':'active'})
    category = []
    for i in soup:
        try:
            d = {'Category': '', 'URL': ''}
            d['Category'] = i.a.text
            d['URL'] = i.a['href']
            category.append(d)
        except:
            print('Error')
    return category


def aconcept_header(url):
    '''
    
    '''
    driver.get(url)
    time.sleep(5)
    try:
        button = driver.find_element_by_id('load-more-groups')
        while button.get_attribute("style") != 'display: none;':
            button.click()
            print('Click Showmore at ' + url)
            time.sleep(10)
    except:
        print('No Showmore button at ' + url)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
df = pd.DataFrame(aconcept_category())
df = df[df['URL'].str.contains('categories')]
df = df.groupby('URL').count()
df

,Category
URL,
https://aconcept-vn.com/en/categories/ban-an,2
https://aconcept-vn.com/en/categories/ban-cafe,2
https://aconcept-vn.com/en/categories/ban-lam-viec,2
https://aconcept-vn.com/en/categories/chan-nen,2
https://aconcept-vn.com/en/categories/den,2
https://aconcept-vn.com/en/categories/do-trang-tri-khac,2
https://aconcept-vn.com/en/categories/ghe-bar,2
https://aconcept-vn.com/en/categories/ghe-bar-1Hf4Acqv,2
https://aconcept-vn.com/en/categories/ghe-don,2


In [5]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()  

#Insert into Category Table
def insert_to_categories(website, category, category_url, product, image, price, url):
    '''
    
    '''
    query = f'''
    INSERT INTO ProductHeader(website, category, category_url, product, image, price, url)
    VALUES('{website}', '{category}', '{category_url}', '{product}', '{image}', '{price}', '{url}');
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)

#Function to delete table record
def delete_from_categories(idx):
    '''
    
    '''
    query = f'''
    DELETE FROM ProductHeader
    WHERE id = {idx};
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY Delete:', err)

        
#insert_product(1, 123, 'Sofa Brand', '["name1", "name2", "name3"]', 'description', 2500000, 'Aconcept', 'Red', 'Leather', 'Contem', 5, 'Sofa', 'minhtran.vn')
def insert_product(id, sku, product, image, description, price, brand, color, materials, style, review, category, url):
    '''
    
    '''
    query = f'''
      INSERT INTO ProductDetail (id, sku, product, image, description, 
                            price, brand, color, materials, style, review, category, url)
      VALUES ('{id}', '{sku}', '{product}', '{image}', '{description}', '{price}', '{brand}', 
      '{color}', '{materials}', '{style}', '{review}', '{category}', '{url}');
      '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)
        
#Function to delete table record
def delete_product(idx):
    '''
    
    '''
    query = f'''
    DELETE FROM ProductDetail
    WHERE id = {idx};
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY Delete:', err)

In [6]:
for category_url in df.index:
    soup = aconcept_header(category_url)
    product_soup = soup.find_all('div', {'class': 'col-md-4 col-sm-6'})
    for i in product_soup:   
        d ={}
        try:
            website = 'aconcept-vn.com'
            category = soup.body.find('section',{'id': 'bannertop'}).h1.text
            category_url = category_url
            product = i.find_all('p')[1].text
            image = i.find('img', {'data-lazy-type': 'image'})['data-lazy-src']
            price = i.find('span', class_=lambda x: x != 'new')
            url = i.a['href']
            insert_to_categories(website, category, category_url, product, image, price, url)
        except:
            print('Error at ' + url)    
    time.sleep(3)
    

Click Showmore at https://aconcept-vn.com/en/categories/ban-an
Click Showmore at https://aconcept-vn.com/en/categories/ban-an
Click Showmore at https://aconcept-vn.com/en/categories/ban-cafe
Click Showmore at https://aconcept-vn.com/en/categories/ban-cafe
Click Showmore at https://aconcept-vn.com/en/categories/ban-cafe
Click Showmore at https://aconcept-vn.com/en/categories/ban-cafe
No Showmore button at https://aconcept-vn.com/en/categories/ban-lam-viec
Click Showmore at https://aconcept-vn.com/en/categories/chan-nen
Click Showmore at https://aconcept-vn.com/en/categories/chan-nen
Click Showmore at https://aconcept-vn.com/en/categories/chan-nen
Click Showmore at https://aconcept-vn.com/en/categories/den
Click Showmore at https://aconcept-vn.com/en/categories/den
Click Showmore at https://aconcept-vn.com/en/categories/den
Click Showmore at https://aconcept-vn.com/en/categories/den
Click Showmore at https://aconcept-vn.com/en/categories/do-trang-tri-khac
Click Showmore at https://aconce

In [7]:
category_url = 'https://aconcept-vn.com/en/categories/ban-lam-viec'
soup = aconcept_header(category_url)
product_soup = soup.find_all('div', {'class': 'col-md-4 col-sm-6'})
for i in product_soup:   
    try:
        website = 'aconcept-vn.com'
        category = soup.body.find('section',{'id': 'bannertop'}).h1.text
        category_url = category_url
        product = i.find_all('p')[1].text
        image = i.find('img', {'data-lazy-type': 'image'})['data-lazy-src']
        price = i.find('span', class_=lambda x: x != 'new')
        url = i.a['href']
        insert_to_categories(website, category, category_url, product, image, price, url)
    except:
        print('Error')

No Showmore button at https://aconcept-vn.com/en/categories/ban-lam-viec


In [13]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

def query_product_url():
    '''
    
    '''
    query = '''
    SELECT a.id, a.url
    FROM project.ProductHeader a
    LEFT JOIN ProductDetail b ON a.id = b.id WHERE b.id is null AND website = 'aconcept-vn.com';
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchall()
    return result

#Insert into Product Table
def insert_to_product(id, product, image, description, price, category, url):
    '''
    
    '''
    query = f'''
    INSERT INTO ProductDetail(id, product, image, description, price, category, url)
    VALUES("{id}", "{product}", '{image}', "{description}", "{price}", "{category}", "{url}");
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)

In [14]:
df = pd.DataFrame(query_product_url(), columns=['id', 'url'])
df = df.set_index('id')

In [16]:
def get_image(soup):
    d = {}
    img_soup = soup.find_all('a', {'class': 'downloadimg'})
    for i in range(len(img_soup)):
        img = img_soup[i]['href']
        d[i] = img
    return d

In [17]:
for idx, url in df['url'].iteritems():
    try:
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        product = soup.find('h1', {'class': 'h2'}).text
        description = soup.find('div', {'class': 'info'}).div.text
        price = soup.find('div', {'class': 'price'}).text
        image = get_image(soup)
        category = 'aconcept'
        spec = ''
        insert_to_product(idx, product, json.dumps(image), description, price, category, url)
        time.sleep(1)
    except:
        print(url)

https://aconcept-vn.com/en/products/decoration-material11?sku=650001335
https://aconcept-vn.com/en/products/fauna?sku=650000599
https://aconcept-vn.com/en/products/edge1?sku=650000009
https://aconcept-vn.com/en/products/chifa?sku=650000007
